<a href="https://colab.research.google.com/github/cayamada/NMR_plots/blob/main/Plot_csp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
from google.colab import files
from matplotlib.font_manager import FontProperties

# Carregar os dados do CSV em um DataFrame
file_path = 'shift_changes.csv'
df = pd.read_csv(file_path)

# Coluna com valores no estilo de resíduos
coluna_residuos = 'Assign F1'

# Filtrar resíduos que terminam com 'H'
pattern = re.compile(r'.+H$')
df_filtrado = df[df[coluna_residuos].str.match(pattern)]

# Remover duplicatas
df_filtrado = df_filtrado.drop_duplicates(subset=[coluna_residuos])

# Extrair os números dos valores filtrados
df_filtrado['Numeros'] = df_filtrado[coluna_residuos].apply(lambda x: int(re.search(r'\d+', x).group()))

# Adicionar um pequeno ruído aos valores do eixo x
#noise = np.random.uniform(-0.1, 0.1, len(df_filtrado))
#df_filtrado['Numeros'] = df_filtrado['Numeros'] + noise

# Escolha as colunas que você deseja plotar
columns_to_plot = ['Numeros', 'Fit Param A']

# Calcular estatísticas
median_value = df_filtrado[columns_to_plot[1]].median()
mean_value = df_filtrado[columns_to_plot[1]].mean()
std_deviation = df_filtrado[columns_to_plot[1]].std()

# Inicializar listas para armazenar os resíduos separados por condições
acima_media_desvio = []
acima_mediana = []
outras_condicoes = []

# Calcular valores a 0.5 e 1.5 desvios padrão
valor_0_5_desvios = mean_value + (0.5 * std_deviation)
valor_1_5_desvios = mean_value + (1.5 * std_deviation)

# Verificar cada valor de perturbação química e separar os resíduos
for i, value in enumerate(df_filtrado['Fit Param A']):
    if value > valor_1_5_desvios:
        acima_media_desvio.append(df_filtrado['Numeros'].iloc[i])
    elif value > valor_0_5_desvios:
        acima_mediana.append(df_filtrado['Numeros'].iloc[i])
    else:
        outras_condicoes.append(df_filtrado['Numeros'].iloc[i])

# Define cores com base em condições
colors = [
    '#F71115' if value > (valor_1_5_desvios) else    # Red: Acima da média + desvio padrão
    '#F4B2B3' if value > valor_0_5_desvios else                    # Azul: Acima da mediana
    '#999999ff'                                                 # Azul escuro: Outros casos
    for value in df_filtrado[columns_to_plot[1]]
]

# Ajuste a cor dos títulos dos eixos e valores dos eixos
font_properties = FontProperties(weight='bold', size=12)  # Ajuste o tamanho e o peso da fonte

# Ajustar o tamanho do gráfico
plt.figure(figsize=(17, 6))  # Largura x Altura em polegadas

# Ajustar a largura das barras
largura_barras = 1.0  # Aumente ou diminua este valor conforme necessário

# Largura do contorno das barras
contour_width = 0.2
# Definir a cor e espessura das bordas das barras
cor_bordas = 'black'
espessura_bordas = 1

# Crie o gráfico de barras
plt.bar(df_filtrado[columns_to_plot[0]], df_filtrado[columns_to_plot[1]], color=colors, edgecolor=cor_bordas, linewidth=espessura_bordas, width=largura_barras)

# Ajustar a espessura das bordas do gráfico
ax = plt.gca()
ax.spines['top'].set_linewidth(2)    # Borda superior
ax.spines['bottom'].set_linewidth(2) # Borda inferior
ax.spines['left'].set_linewidth(2)   # Borda esquerda
ax.spines['right'].set_linewidth(2)  # Borda direita

# Adicione rótulos e título
plt.xlabel('Resíduos', fontproperties=font_properties)
plt.ylabel('Δδ (ppm)', fontproperties=font_properties)
plt.axhline(y=valor_0_5_desvios, color='black', linestyle='--', label='0.5 σ')
plt.axhline(y=valor_1_5_desvios, color='black', linestyle=':', label='1.5 σ')

# Adicione linhas para median, média e desvio padrão
#plt.axhline(y=median_value, color='black', linestyle='-', label='Mediana')
#plt.axhline(y=mean_value, color='black', linestyle='--', label='Média')
#plt.axhline(y=mean_value + std_deviation, color='black', linestyle='--', label='Média +/- Desvio Padrão')
#plt.axhline(y=mean_value - std_deviation, color='black', linestyle='--') #, label='Média - Desvio Padrão')

# Adicione legenda
plt.legend()

# Salvar a imagem temporariamente no Colab
plt.savefig('grafico_temp.png', dpi=300, bbox_inches='tight')

# Exiba o gráfico
plt.show()

In [ ]:
# Imprimir os resíduos separados por condições
print("Resíduos acima da média + desvio padrão:", sorted(acima_media_desvio))
print("Resíduos acima da mediana:", sorted(acima_mediana))
print("Outros resíduos:", sorted(outras_condicoes))